Process the raw output file, by removing header lines and spaces

In [2]:
with open('../../output/output.csv', 'r') as file:
    lines = file.readlines()

# Remove all spaces in all rows
lines = [line.replace(' ', '') for line in lines]

# Remove lines that are the same as the first one
header = lines[0]
lines = [line for line in lines[1:] if line != lines[0]]

# Join the lines back into a single string
output = ''.join(lines)

# Save the output to a file
with open('preprocessed.csv', 'w') as file:
    file.write(header)
    file.write(output)


Process the data in pandas

In [5]:
import pandas as pd

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('preprocessed_without_nan.csv')
df = df[df['time'] != 0]
df.drop('run_id', axis=1, inplace=True)

# Display the DataFrame
df.to_csv('preprocessed2.csv', index=False)


In [4]:
# check the data types
df.dtypes

bin_name                      object
config_name                   object
const_basicHintCount           int64
const_enableBasicHead          int64
const_enableDense              int64
const_enableDense2             int64
const_enableDensifySplit       int64
const_enableHash               int64
const_enableHashAdapt          int64
const_enableHeadNode           int64
const_enablePrefix             int64
const_hashSimdWidth            int64
const_hashSortUseStdMerge      int64
const_hashUseCrc32             int64
const_hashUseSimd              int64
const_headNode4HintCount       int64
const_headNode8HintCount       int64
const_pageSizeInner            int64
const_pageSizeLeaf             int64
data_name                     object
data_size                      int64
data_sorted                    int64
density                      float64
op                            object
payload_size                   int64
rand_seed                      int64
run_id                        object
y